# Labelling using Colab [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/dafrie/fin-disclosures-nlp/blob/master/notebooks/ColabLabelling.ipynb)

- Uses custom widgets for labelling
- Loads and stores the data in a google storage bucket

